# Acoustic Model Zoo demo

In [1]:
import torch
import torchaudio
import IPython.display as display
from acoustic.model import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
hubert   = torch.hub.load("bshall/hubert:main",          "hubert_soft").to(device)
hifigan  = torch.hub.load("bshall/hifigan:main", "hifigan_hubert_soft").to(device)

Using cache found in C:\Users\jiangsongru/.cache\torch\hub\bshall_hubert_main
Using cache found in C:\Users\jiangsongru/.cache\torch\hub\bshall_hifigan_main


Removing weight norm...


In [6]:
dataset = 'databaker'

list = f'{dataset}-full'
#list = f'{dataset}-4h'
#list = f'{dataset}-2h'
#list = f'{dataset}-1h'
#list = f'{dataset}-30min'
#list = f'{dataset}-10min'

#model = 'baseline'
#model = 'no_dropout'
#model = 'no_IN'
#model = 'single_LSTM'
#model = 'only_Encoder'
#model = 'only_Decoder'
model = 'tiny'
#model = 'tiny_half'

steps = 'best'
#steps = 7000

acoustic = globals().get(f'AcousticModel_{model}', 'AcousticModel')().to(device)
ckpt = torch.load(f'log/{model}_{list}/model-{steps}.pt', map_location=device)
acoustic.load_state_dict(ckpt["acoustic-model"])
print('this ckpt is at step:', ckpt['step'])

this ckpt is at step: 36000


In [7]:
## speech
wav_fp = 'test/LJ001-0001.wav'
#wav_fp = 'test/000001.wav'
#wav_fp = 'test/BAC009S0724W0121.wav'
#wav_fp = 'test/SSB00050353.wav'
#wav_fp = 'test/common_voice_ab_19904194.wav'
#wav_fp = 'test/000df995-51ab-4a1d-852c-c288e0300bf7.wav'
## vocal (recllst)
#wav_fp = 'test/D#4_しゅしゅししょしゃしょし.wav'
#wav_fp = 'test/_べべぶぼべぼぼ.wav'
#wav_fp = 'test/_きゅんきゅきゅきぇきゅきょきゅ.wav'
#wav_fp = 'test/kan_kan_kan.wav'
## vocal (song)
#wav_fp = 'test/[Hana] FLOWER_1_歌声-トラック_tune-8.wav'
#wav_fp = 'test/[Ema] さがり花_1_歌声-トラック-6.wav'
#wav_fp = 'test/[Len] テロル_vocal-4.wav'
## instrumental
#wav_fp = 'test/Ⅱ 小快板-14.wav'
#wav_fp = 'test/十二音小夜曲-10.wav'

source, sr = torchaudio.load(wav_fp)
source = torchaudio.functional.resample(source, sr, 16000)
source = source.unsqueeze(0).to(device)

print('original:')
display.Audio(source.squeeze().cpu(), rate=16000)

original:


In [8]:
with torch.inference_mode():
  # Extract speech units
  units = hubert.units(source)
  # Generate target spectrogram
  mel = acoustic.generate(units).transpose(1, 2)
  # Generate audio waveform
  target = hifigan(mel)

print('converted:')
display.Audio(target.squeeze().cpu(), rate=16000)

converted:
